In [1]:
#!pip install dash


In [2]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


In [3]:
#!pip install plotly --upgrade

In [4]:
!curl -o spacex_launch_dash.csv "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
   

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  2476  100  2476    0     0   5513      0 --:--:-- --:--:-- --:--:--  5539


In [5]:
#!curl -o spacex_dash_app.py https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py
    

In [6]:
# Create a dash application
app = dash.Dash(__name__)

In [7]:
# TASK 1: Add a dropdown list to enable Launch Site selection
# Get a list of unique launch sites

#launch_sites = spacex_df['Launch Site'].unique().tolist()

In [8]:
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
#spacex_df.head()

In [9]:
# Correctly obtain unique launch sites
launch_sites = spacex_df['Launch Site'].unique().tolist()

# Correctly prepare the options for the dropdown
options_list =[{'label': 'ALL', 'value': 'ALL'}] + [{'label': site, 'value': site} for site in launch_sites]


In [10]:
# TASK 2: Add a pie chart to show the total successful launches count for all sites
# If a specific launch site was selected, show the Success vs. Failed counts for the site
#                                 html.Div(dcc.Graph(id='success-pie-chart')),
#                                 html.Br(),

#                                 html.P("Payload range (Kg):"),

In [11]:
app.layout = html.Div(children=[
    html.H1(
        'SpaceX Launch Records Dashboard',
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': 20}
    ),
    
    # Dropdown container
    html.Div([
        dcc.Dropdown(
            id='site-dropdown',
            options=options_list,
            value='ALL',
            clearable=False,
            searchable=True,
            placeholder='Select a Launch Site',
            style={'width': '100%'}
        )
    ], style={'width': '90%', 'margin': '5px auto'}),
    
    
    # RangeSlider and its placeholder
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={i: str(i) for i in range(0, 10000, 1000)},  # Custom marks at every 10 units
        #marks={0: '0', 100: '100'},
        value=[min_payload, max_payload]
    ),
    html.Div(id='slider-output'),  # Placeholder to display selected range
 
    

    # Pie chart container
    html.Div([
        dcc.Graph(id='success-pie-chart', style={'height': '280px'}),
    ], style={'width': '90%', 'margin': '5px auto'}),

    # Scatter plot container
    html.Div([
        dcc.Graph(id='success-payload-scatter-chart', style={'height': '280px'}),
    ], style={'width': '90%', 'margin': '5px auto'}),

], style={'maxWidth': '1200px', 'margin': '0 auto'})


In [12]:
#print(options_list)

In [13]:
#from dash.exceptions import PreventUpdate

@app.callback(
    Output('success-pie-chart', 'figure'),
    [Input('site-dropdown', 'value')])

def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # If 'ALL' sites are selected, show the total successful launches count for all sites
        df_filtered = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
        figure = px.pie(df_filtered, names='Launch Site', values='class', title='Total Successful Launches by Site')
    else:
        # If a specific site is selected, show the Success vs. Failed counts for the site
        df_filtered = spacex_df[spacex_df['Launch Site'] == selected_site]
        status_counts = df_filtered['class'].value_counts().reset_index()
        
        # Rename the indices directly
        status_counts['index'] = status_counts['index'].replace({0: 'Failed', 1: 'Successful'})
        
        figure = px.pie(status_counts, names='index', values='class',
                     title=f'Success vs Failed Launches for {selected_site}')
        print(status_counts['index'])
    
    return figure


In [14]:
# TASK 3: Add a slider to select payload range
#dcc.RangeSlider(id='payload-slider',...)

Next, we want to find if variable payload is correlated to mission outcome. From a dashboard point of view, we
want to be able to easily select different payload range and see if we can identify some visual patterns.

In [15]:
#TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot


Next, we want to plot a scatter plot with the x axis to be the payload and the y axis to be the launch outcome (i.e., class column).
As such, we can visually observe how payload may be correlated with mission outcomes for selected site(s).

In addition, we want to color-label the Booster version on each scatter point so that we may
observe mission outcomes with different boosters.

Now, let’s add a call function including the following application logic:

In [16]:
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)

def update_scatter_chart(selected_site, payload_range):
    # Filter data based on selected site (if not 'ALL') and payload range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
                            (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
    
    # Create scatter chart using plotly express
    fig = px.scatter(filtered_df, 
                     x='Payload Mass (kg)', 
                     y='class', 
                     #color='class',  # This will give different colors for success and fail
                     color="Booster Version Category",
                     labels={'class': 'Launch Success'}, 
                     title='Correlation between Payload and Launch Success')

    # Adjust the y-axis labels (assuming 'class' column has 0 for fail and 1 for success)
    fig.update_yaxes(tickvals=[0, 1], ticktext=['Fail', 'Success'])

    return fig


In [17]:
# Run the app
if __name__ == '__main__':
    app.run_server()
